In [110]:
import pandas as pd

data_path = 'MaStR_Stromerzeuger_Wind_in Betrieb_Inbetriebnahmedatum bis 2010.csv'
df_MaStR = pd.read_csv(data_path, sep=';', decimal=',', dayfirst=True)
df_MaStR = df_MaStR[['Typenbezeichnung',
                     'Bruttoleistung der Einheit',
                     'Rotordurchmesser der Windenergieanlage',
                     'Nabenhöhe der Windenergieanlage',
                     'Hersteller der Windenergieanlage']]
df_MaStR = df_MaStR.dropna()
df_MaStR = df_MaStR.reset_index()
df_MaStR = df_MaStR.drop('index', axis=1)

df_MaStR.rename(columns={'Typenbezeichnung': 'Model MaStR'}, inplace=True)
df_MaStR.rename(columns={'Bruttoleistung der Einheit': 'Nennleistung in kW MaStR'}, inplace=True)
df_MaStR.rename(columns={'Rotordurchmesser der Windenergieanlage': 'Durchmesser in m MaStR'}, inplace=True)
df_MaStR.rename(columns={'Nabenhöhe der Windenergieanlage': 'Nabenhöhe in m MaStR'}, inplace=True)
df_MaStR.rename(columns={'Hersteller der Windenergieanlage': 'Hersteller MaStR'}, inplace=True)

df_MaStR = df_MaStR.drop_duplicates(subset=['Model MaStR', 
                                            'Nennleistung in kW MaStR',
                                            'Durchmesser in m MaStR',
                                            'Nabenhöhe in m MaStR',
                                            'Hersteller MaStR'])

df_MaStR = df_MaStR.dropna()
df_MaStR = df_MaStR.reset_index()
df_MaStR = df_MaStR.drop('index', axis=1)

df_MaStR.head(10)

,Model MaStR,Nennleistung in kW MaStR,Durchmesser in m MaStR,Nabenhöhe in m MaStR,Hersteller MaStR
0,FL1000,1000.0,54.0,70.0,Fuhrländer AG
1,N43,600.0,43.0,60.0,Nordex Energy GmbH
2,V66,1750.0,66.0,67.0,Vestas Deutschland GmbH
3,MM82,2000.0,82.0,59.0,Senvion Deutschland GmbH
4,FL 1000,1000.0,54.0,110.0,Fuhrländer AG
5,TW 600e,600.0,46.0,60.0,Tacke GmbH & Co. KG
6,FUH15 G1 D250,250.0,30.0,42.0,Fuhrländer AG
7,TW600a,600.0,46.0,70.0,Tacke GmbH & Co. KG
8,GE 2.5 xl,2500.0,100.0,100.0,General Electric Deutschland Holding GmbH
9,NTK 300/31,300.0,31.0,31.0,NEG Micon Deutschland GmbH


In [111]:
df_WTM = pd.read_excel("WTM_with_UBA-clusters_modified.xlsx")
df_WTM.columns = df_WTM.iloc[0]
df_WTM = df_WTM.drop(0)
df_WTM = df_WTM.loc[df_WTM['Hersteller'] != '-']
df_WTM = df_WTM[['Model',
                'Nennleistung in kW',
                'Durchmesser in m',
                'Nabenhöhe in m',
                'Hersteller']]
df_WTM = df_WTM.reset_index()
df_WTM = df_WTM.drop('index', axis=1)

df_WTM.head(10)

,Model,Nennleistung in kW,Durchmesser in m,Nabenhöhe in m,Hersteller
0,AAER A1500-77,1500,77,65/85,NOI Rotortechnik
1,ACSA A27/225,225,27,30,LM Glasfieber
2,AMSC wt1650df,1650,82,70/80,Windtec
3,AN Bonus 100/30,100,23,24/30,LM Glasfieber
4,AN Bonus 1000/54,1000,54.2,50/60/70,LM Glasfieber/ Bonus
5,AN Bonus 1300/62,1300,62,68/80/90,LM Glasfieber/ Bonus
6,AN Bonus 150/30,150,23,30/40,LM Glasfieber
7,AN Bonus 2000/76,2000,76,60/80/98,LM Glasfieber/ Bonus
8,AN Bonus 2300/82,2300,82.4,80/90/100,Bonus
9,AN Bonus 300/33,300,33.4,30/40,LM Glasfieber/Bonus


In [113]:
from thefuzz import fuzz, process
from tqdm import tqdm
import numpy as np


df_match = pd.DataFrame()

for i in tqdm(range(len(df_WTM))):

    Hersteller_WTM = df_WTM.loc[i, 'Hersteller']
    df_i = pd.DataFrame()
    for j in range(len(df_MaStR)):

        Hersteller_MaStR = df_MaStR.loc[j, 'Hersteller MaStR']

        ratio = fuzz.ratio(Hersteller_MaStR, Hersteller_WTM)
        partial_ratio = fuzz.partial_ratio(Hersteller_MaStR, Hersteller_WTM)
        token_sort_ratio = fuzz.token_sort_ratio(Hersteller_MaStR, Hersteller_WTM)
        token_set_ratio = fuzz.token_set_ratio(Hersteller_MaStR, Hersteller_WTM)
        similarity = max(ratio,partial_ratio,token_sort_ratio,token_set_ratio)

        if similarity > 50:
                Nennleistung_WTM = df_WTM.loc[i, 'Nennleistung in kW']
                Nennleistung_MaStR = df_MaStR.loc[j, 'Nennleistung in kW MaStR']
                if Nennleistung_WTM == Nennleistung_MaStR:
                    WTM = pd.DataFrame(df_WTM.loc[i]).transpose()
                    MaStR = pd.DataFrame(df_MaStR.loc[j]).transpose()
                    wtm_values  = WTM.values[0]
                    mastr_values  = MaStR.values[0]
                    combined_values = np.concatenate([wtm_values , mastr_values ])
                    df_j = pd.DataFrame([combined_values], columns=WTM.columns.tolist() + MaStR.columns.tolist())
                    df_i = pd.concat([df_i, df_j])
    df_match = pd.concat([df_match, df_i])


 46%|████▋     | 387/834 [17:57<20:44,  2.79s/it]  


KeyboardInterrupt: 

In [ ]:
df_match.to_excel('Zwischenergebnis.xlsx')

,Model,Nennleistung in kW,Durchmesser in m,Nabenhöhe in m,Hersteller,Model MaStR,Nennleistung in kW MaStR,Durchmesser in m MaStR,Nabenhöhe in m MaStR,Hersteller MaStR
0,AN Bonus 1000/54,1000.0,54.2,50/60/70,LM Glasfieber/ Bonus,AN Bonus 1MW/54,1000.0,54.0,60.0,Bonus Energy A/S
0,AN Bonus 1000/54,1000.0,54.2,50/60/70,LM Glasfieber/ Bonus,1MW,1000.0,54.0,60.0,Bonus Energy A/S
0,AN Bonus 1000/54,1000.0,54.2,50/60/70,LM Glasfieber/ Bonus,AN Bonus 1 MW/54,1000.0,54.0,60.0,Bonus Energy A/S
0,AN Bonus 1000/54,1000.0,54.2,50/60/70,LM Glasfieber/ Bonus,AN Bonus 1 MW/54,1000.0,54.0,64.0,Bonus Energy A/S
0,AN Bonus 1300/62,1300.0,62.0,68/80/90,LM Glasfieber/ Bonus,AN Bonus 62,1300.0,62.0,80.0,Bonus Energy A/S
...,...,...,...,...,...,...,...,...,...,...
0,eno energy eno 82 1.5,1500.0,82.4,58.6 / 86,LM Wind Power,"TW 1,5s",1500.0,70.0,65.0,GE Wind Energy GmbH
0,eno energy eno 82 1.5,1500.0,82.4,58.6 / 86,LM Wind Power,MD70,1500.0,70.0,65.0,GE Wind Energy GmbH
0,eno energy eno 82 1.5,1500.0,82.4,58.6 / 86,LM Wind Power,"GE - 1,5 sl",1500.0,77.0,61.4,GE Wind Energy GmbH
0,eno energy eno 82 1.5,1500.0,82.4,58.6 / 86,LM Wind Power,GE WE 1.5csl,1500.0,77.0,100.0,GE Wind Energy GmbH


In [ ]:
df_match = pd.read_excel('Zwischenergebnis.xlsx')
df_match = df_match.reset_index()
df_match = df_match.drop('index', axis=1)

for i in tqdm(range(len(df_match))):
    Durchmesser_WTM = df_match.loc[i, 'Durchmesser in m']
    Durchmesser_MaStR = df_match.loc[i, 'Durchmesser in m MaStR']
    x = Durchmesser_WTM/Durchmesser_MaStR
    if x > 1.05:
        df_match.drop(i, inplace=True)
        continue
    if x < 0.95:
        df_match.drop(i, inplace=True)
        continue

df_match = df_match.reset_index()
df_match = df_match.drop('index', axis=1)
df_match

  0%|          | 0/9252 [00:00<?, ?it/s]

100%|██████████| 9252/9252 [00:15<00:00, 582.01it/s] 


,Model,Nennleistung in kW,Durchmesser in m,Nabenhöhe in m,Hersteller,Model MaStR,Nennleistung in kW MaStR,Durchmesser in m MaStR,Nabenhöhe in m MaStR,Hersteller MaStR
0,AN Bonus 1000/54,1000.0,54.2,50/60/70,LM Glasfieber/ Bonus,AN Bonus 1MW/54,1000.0,54.0,60.0,Bonus Energy A/S
1,AN Bonus 1000/54,1000.0,54.2,50/60/70,LM Glasfieber/ Bonus,1MW,1000.0,54.0,60.0,Bonus Energy A/S
2,AN Bonus 1000/54,1000.0,54.2,50/60/70,LM Glasfieber/ Bonus,AN Bonus 1 MW/54,1000.0,54.0,60.0,Bonus Energy A/S
3,AN Bonus 1000/54,1000.0,54.2,50/60/70,LM Glasfieber/ Bonus,AN Bonus 1 MW/54,1000.0,54.0,64.0,Bonus Energy A/S
4,AN Bonus 1300/62,1300.0,62.0,68/80/90,LM Glasfieber/ Bonus,AN Bonus 62,1300.0,62.0,80.0,Bonus Energy A/S
...,...,...,...,...,...,...,...,...,...,...
2805,Windey WD50-750,750.0,50.0,50/65,LM Glasfiber A/S,ww 750/52,750.0,52.0,73.8,Wind World A/S
2806,Windey WD50-750,750.0,50.0,50/65,LM Glasfiber A/S,WW 750,750.0,52.0,74.0,Wind World A/S
2807,Windey WD50-750,750.0,50.0,50/65,LM Glasfiber A/S,Windworld 750,750.0,52.0,74.0,Wind World A/S
2808,Windey WD50-750,750.0,50.0,50/65,LM Glasfiber A/S,WW750,750.0,52.0,74.0,Wind World A/S


In [ ]:
import re

df = pd.DataFrame()

for i in tqdm(range(len(df_match))):
    input_string = df_match['Nabenhöhe in m'][i]
    numbers = re.findall(r'\b\d+\b', input_string)    
    number_list = [int(number) for number in numbers] 
    for number in number_list:
        df_i = df_match.iloc[i]
        df_i = df_i.to_frame(name=(i))
        df_i = df_i.transpose()
        df_i['Nabenhöhe in m'][i] = number
        df = pd.concat([df,df_i])
df_match = df.reset_index()
df_match = df_match.drop('index', axis=1)

df_match

  0%|          | 0/2810 [00:00<?, ?it/s]

100%|██████████| 2810/2810 [00:54<00:00, 51.58it/s] 


,Model,Nennleistung in kW,Durchmesser in m,Nabenhöhe in m,Hersteller,Model MaStR,Nennleistung in kW MaStR,Durchmesser in m MaStR,Nabenhöhe in m MaStR,Hersteller MaStR
0,AN Bonus 1000/54,1000.0,54.2,50,LM Glasfieber/ Bonus,AN Bonus 1MW/54,1000.0,54.0,60.0,Bonus Energy A/S
1,AN Bonus 1000/54,1000.0,54.2,60,LM Glasfieber/ Bonus,AN Bonus 1MW/54,1000.0,54.0,60.0,Bonus Energy A/S
2,AN Bonus 1000/54,1000.0,54.2,70,LM Glasfieber/ Bonus,AN Bonus 1MW/54,1000.0,54.0,60.0,Bonus Energy A/S
3,AN Bonus 1000/54,1000.0,54.2,50,LM Glasfieber/ Bonus,1MW,1000.0,54.0,60.0,Bonus Energy A/S
4,AN Bonus 1000/54,1000.0,54.2,60,LM Glasfieber/ Bonus,1MW,1000.0,54.0,60.0,Bonus Energy A/S
...,...,...,...,...,...,...,...,...,...,...
11886,Windey WD50-750,750.0,50.0,50,LM Glasfiber A/S,WW750,750.0,52.0,74.0,Wind World A/S
11887,Windey WD50-750,750.0,50.0,65,LM Glasfiber A/S,WW750,750.0,52.0,74.0,Wind World A/S
11888,eno energy eno 82 1.5,1500.0,82.4,58,LM Wind Power,GE Wind 1.5 sl,1500.0,80.0,96.0,GE Wind Energy GmbH
11889,eno energy eno 82 1.5,1500.0,82.4,6,LM Wind Power,GE Wind 1.5 sl,1500.0,80.0,96.0,GE Wind Energy GmbH


In [ ]:
for i in tqdm(range(len(df_match))):
    Nabenhöhe_WTM = df_match.loc[i, 'Nabenhöhe in m']
    Nabenhöhe_MaStR = df_match.loc[i, 'Nabenhöhe in m MaStR']
    x = Nabenhöhe_WTM/Nabenhöhe_MaStR
    if x > 1.05:
        df_match.drop(i, inplace=True)
        continue
    if x < 0.95:
        df_match.drop(i, inplace=True)
        continue

df_match = df_match.reset_index()
df_match = df_match.drop('index', axis=1)
df_match

100%|██████████| 2492/2492 [00:00<00:00, 27382.58it/s]


,Model,Nennleistung in kW MaStR,Nennleistung in kW,Durchmesser in m MaStR,Durchmesser in m,Nabenhöhe in m MaStR,Nabenhöhe in m,Hersteller MaStR,Hersteller
0,AN Bonus 1000/54,1000.0,1000.0,54.0,54.2,60.0,60,Bonus Energy A/S,LM Glasfieber/ Bonus
1,AN Bonus 1000/54,1000.0,1000.0,54.0,54.2,60.0,60,Bonus Energy A/S,LM Glasfieber/ Bonus
2,AN Bonus 1000/54,1000.0,1000.0,54.0,54.2,60.0,60,Bonus Energy A/S,LM Glasfieber/ Bonus
3,AN Bonus 1300/62,1300.0,1300.0,62.0,62.0,80.0,80,Bonus Energy A/S,LM Glasfieber/ Bonus
4,AN Bonus 1300/62,1300.0,1300.0,62.0,62.0,68.0,68,Bonus Energy A/S,LM Glasfieber/ Bonus
...,...,...,...,...,...,...,...,...,...
2487,Wind World WW 750/52,750.0,750.0,52.0,52.0,73.8,73,Wind World A/S,LM Glasfiber A/S
2488,Wind World WW 750/52,750.0,750.0,52.0,52.0,74.0,73,Wind World A/S,LM Glasfiber A/S
2489,Wind World WW 750/52,750.0,750.0,52.0,52.0,74.0,73,Wind World A/S,LM Glasfiber A/S
2490,Wind World WW 750/52,750.0,750.0,52.0,52.0,74.0,73,Wind World A/S,LM Glasfiber A/S


In [377]:
df_match = pd.read_excel('Match_Step4.xlsx')
df_match = df_match[['Model MaStR',
                     'Model',
                     'Nennleistung in kW MaStR',
                     'Nennleistung in kW',
                     'Durchmesser in m MaStR',
                     'Durchmesser in m',
                     'Nabenhöhe in m MaStR',
                     'Nabenhöhe in m',
                     'Hersteller MaStR',
                     'Hersteller']]

df_match

,Model MaStR,Model,Nennleistung in kW MaStR,Nennleistung in kW,Durchmesser in m MaStR,Durchmesser in m,Nabenhöhe in m MaStR,Nabenhöhe in m,Hersteller MaStR,Hersteller
0,AN Bonus 1MW/54,AN Bonus 1000/54,1000,1000,54.0,54.2,60.0,60,Bonus Energy A/S,LM Glasfieber/ Bonus
1,1MW,AN Bonus 1000/54,1000,1000,54.0,54.2,60.0,60,Bonus Energy A/S,LM Glasfieber/ Bonus
2,AN Bonus 1 MW/54,AN Bonus 1000/54,1000,1000,54.0,54.2,60.0,60,Bonus Energy A/S,LM Glasfieber/ Bonus
3,AN Bonus 62,AN Bonus 1300/62,1300,1300,62.0,62.0,80.0,80,Bonus Energy A/S,LM Glasfieber/ Bonus
4,"AN Bonus 1,3 MW/62",AN Bonus 1300/62,1300,1300,62.0,62.0,68.0,68,Bonus Energy A/S,LM Glasfieber/ Bonus
...,...,...,...,...,...,...,...,...,...,...
2487,ww 750/52,Wind World WW 750/52,750,750,52.0,52.0,73.8,73,Wind World A/S,LM Glasfiber A/S
2488,WW 750,Wind World WW 750/52,750,750,52.0,52.0,74.0,73,Wind World A/S,LM Glasfiber A/S
2489,Windworld 750,Wind World WW 750/52,750,750,52.0,52.0,74.0,73,Wind World A/S,LM Glasfiber A/S
2490,WW750,Wind World WW 750/52,750,750,52.0,52.0,74.0,73,Wind World A/S,LM Glasfiber A/S


In [284]:
df_match['Schlüssel'] = '-'

for i in tqdm(range(len(df_match))):
    Index = (str(df_match['Model MaStR'][i]) + ', ' + 
            str(df_match['Nennleistung in kW MaStR'][i]) + ', ' + 
            str(df_match['Durchmesser in m MaStR'][i]) + ', ' +
            str(df_match['Nabenhöhe in m MaStR'][i]) + ', ' +
            str(df_match['Hersteller MaStR'][i]))
    df_match.loc[i, 'Schlüssel'] = Index


df_match = df_match.set_index('Schlüssel')
df_match

  0%|          | 0/2492 [00:00<?, ?it/s]

100%|██████████| 2492/2492 [00:00<00:00, 4773.90it/s]


,Model MaStR,Model,Nennleistung in kW MaStR,Nennleistung in kW,Durchmesser in m MaStR,Durchmesser in m,Nabenhöhe in m MaStR,Nabenhöhe in m,Hersteller MaStR,Hersteller
Schlüssel,,,,,,,,,,
"AN Bonus 1MW/54, 1000, 54.0, 60.0, Bonus Energy A/S",AN Bonus 1MW/54,AN Bonus 1000/54,1000,1000,54.0,54.2,60.0,60,Bonus Energy A/S,LM Glasfieber/ Bonus
"1MW, 1000, 54.0, 60.0, Bonus Energy A/S",1MW,AN Bonus 1000/54,1000,1000,54.0,54.2,60.0,60,Bonus Energy A/S,LM Glasfieber/ Bonus
"AN Bonus 1 MW/54, 1000, 54.0, 60.0, Bonus Energy A/S",AN Bonus 1 MW/54,AN Bonus 1000/54,1000,1000,54.0,54.2,60.0,60,Bonus Energy A/S,LM Glasfieber/ Bonus
"AN Bonus 62, 1300, 62.0, 80.0, Bonus Energy A/S",AN Bonus 62,AN Bonus 1300/62,1300,1300,62.0,62.0,80.0,80,Bonus Energy A/S,LM Glasfieber/ Bonus
"AN Bonus 1,3 MW/62, 1300, 62.0, 68.0, Bonus Energy A/S","AN Bonus 1,3 MW/62",AN Bonus 1300/62,1300,1300,62.0,62.0,68.0,68,Bonus Energy A/S,LM Glasfieber/ Bonus
...,...,...,...,...,...,...,...,...,...,...
"ww 750/52, 750, 52.0, 73.8, Wind World A/S",ww 750/52,Wind World WW 750/52,750,750,52.0,52.0,73.8,73,Wind World A/S,LM Glasfiber A/S
"WW 750, 750, 52.0, 74.0, Wind World A/S",WW 750,Wind World WW 750/52,750,750,52.0,52.0,74.0,73,Wind World A/S,LM Glasfiber A/S
"Windworld 750, 750, 52.0, 74.0, Wind World A/S",Windworld 750,Wind World WW 750/52,750,750,52.0,52.0,74.0,73,Wind World A/S,LM Glasfiber A/S


In [285]:
df_match.to_excel('Zwischenergebnis.xlsx')

In [380]:
df_match = pd.read_excel('Zwischenergebnis.xlsx')
df_match = df_match.set_index('Schlüssel')
df_new = pd.DataFrame()
modelle = df_match.index.to_list()
print(len(df_match))
for m in tqdm(modelle):
    df_i = df_match.loc[m]
    if isinstance(df_i, pd.Series):
        continue
    df_i = df_i.reset_index()
    max_similarity = 0

    for j in range(len(df_i)):
        Model = df_i.loc[0, 'Model MaStR']
        Model_j = df_i.loc[j, 'Model']

        ratio = fuzz.ratio(Model, Model_j)
        partial_ratio = fuzz.partial_ratio(Model, Model_j)
        token_sort_ratio = fuzz.token_sort_ratio(Model, Model_j)
        token_set_ratio = fuzz.token_set_ratio(Model, Model_j)
        similarity = max(ratio,partial_ratio,token_sort_ratio,token_set_ratio)
        if similarity > max_similarity:
            max_similarity = similarity
            keep = j

    for k in range(len(df_i)):
        if k != keep:
            df_i.drop(k, inplace=True)
        
    df_new = pd.concat([df_new, df_i])
    
df_result = df_new.set_index('Schlüssel')
df_result

2492


100%|██████████| 2492/2492 [00:07<00:00, 333.47it/s]


,Model MaStR,Model,Nennleistung in kW MaStR,Nennleistung in kW,Durchmesser in m MaStR,Durchmesser in m,Nabenhöhe in m MaStR,Nabenhöhe in m,Hersteller MaStR,Hersteller
Schlüssel,,,,,,,,,,
"1.3MW/62, 1300, 62.0, 68.0, Bonus Energy A/S",1.3MW/62,AN Bonus 1300/62,1300,1300,62.0,62.0,68.0,68,Bonus Energy A/S,LM Glasfieber/ Bonus
"1.3MW/62, 1300, 62.0, 68.0, Bonus Energy A/S",1.3MW/62,AN Bonus 1300/62,1300,1300,62.0,62.0,68.0,68,Bonus Energy A/S,LM Glasfieber/ Bonus
"1300, 1300, 62.0, 68.0, Bonus Energy A/S",1300,AN Bonus 1300/62,1300,1300,62.0,62.0,68.0,68,Bonus Energy A/S,LM Glasfieber/ Bonus
"1300, 1300, 62.0, 68.0, Bonus Energy A/S",1300,AN Bonus 1300/62,1300,1300,62.0,62.0,68.0,68,Bonus Energy A/S,LM Glasfieber/ Bonus
"1MW, 1000, 54.0, 60.0, Bonus Energy A/S",1MW,AN Bonus 1000/54,1000,1000,54.0,54.2,60.0,60,Bonus Energy A/S,LM Glasfieber/ Bonus
...,...,...,...,...,...,...,...,...,...,...
"ww52/750, 750, 52.0, 73.9, Wind World A/S",ww52/750,Seewind S 52 750,750,750,52.0,52.0,73.9,73,Wind World A/S,LM Glasfiber A/S
"WW750, 750, 52.0, 74.0, Wind World A/S",WW750,Wind World WW 750/52,750,750,52.0,52.0,74.0,73,Wind World A/S,LM Glasfiber A/S
"WW750, 750, 52.0, 74.0, Wind World A/S",WW750,Wind World WW 750/52,750,750,52.0,52.0,74.0,73,Wind World A/S,LM Glasfiber A/S


In [294]:
df_match = pd.read_excel('Zwischenergebnis.xlsx')
df_match = df_match.set_index('Schlüssel')
df_result2 = pd.DataFrame()
modelle = df_match.index.to_list()
print(len(df_match))
for m in tqdm(modelle):
    df_i = df_match.loc[m]
    if isinstance(df_i, pd.Series) ==False:
        continue
    df_i = pd.DataFrame(df_i).T
    max_similarity = 0

    df_result2 = pd.concat([df_result2, df_i])

df_result2.index.name = 'Schlüssel'
df_result2

2492


100%|██████████| 2492/2492 [00:03<00:00, 829.84it/s] 


,Model MaStR,Model,Nennleistung in kW MaStR,Nennleistung in kW,Durchmesser in m MaStR,Durchmesser in m,Nabenhöhe in m MaStR,Nabenhöhe in m,Hersteller MaStR,Hersteller
Schlüssel,,,,,,,,,,
"EasyWind6AC nennleistung6KW, 6, 6.0, 19.0, Easywind GmbH",EasyWind6AC nennleistung6KW,EasyWind EasyWind 6.0,6,6,6.0,6.0,19.0,19,Easywind GmbH,EasyWind
"Easy Wind 6 AC, 6, 6.0, 19.0, Easywind GmbH",Easy Wind 6 AC,EasyWind EasyWind 6.0,6,6,6.0,6.0,19.0,19,Easywind GmbH,EasyWind
"6AC, 6, 6.0, 19.0, Easywind GmbH",6AC,EasyWind EasyWind 6.0,6,6,6.0,6.0,19.0,19,Easywind GmbH,EasyWind
"EasyWind 6 AC, 6, 6.0, 19.0, Easywind GmbH",EasyWind 6 AC,EasyWind EasyWind 6.0,6,6,6.0,6.0,19.0,19,Easywind GmbH,EasyWind
"6 AC, 6, 6.0, 19.0, Easywind GmbH",6 AC,EasyWind EasyWind 6.0,6,6,6.0,6.0,19.0,19,Easywind GmbH,EasyWind
...,...,...,...,...,...,...,...,...,...,...
"W-2700 150 kW, 150, 27.0, 41.0, Wind World A/S",W-2700 150 kW,Wind World W2700,150,150,27.0,27.0,41.0,41,Wind World A/S,LM Glaasfiber A/S
"W-2700, 150, 27.0, 31.0, Wind World A/S",W-2700,Wind World W2700,150,150,27.0,27.0,31.0,31,Wind World A/S,LM Glaasfiber A/S
"W2700, 150, 27.0, 31.0, Wind World A/S",W2700,Wind World W2700,150,150,27.0,27.0,31.0,31,Wind World A/S,LM Glaasfiber A/S


In [381]:
df_final = pd.concat([df_result, df_result2])
df_final = df_final.drop_duplicates()

In [382]:
data_path = 'MaStR_Stromerzeuger_Wind_in Betrieb_Inbetriebnahmedatum bis 2010.csv'
df_MaStR = pd.read_csv(data_path, sep=';', decimal=',', dayfirst=True)
df_MaStR = df_MaStR[['Typenbezeichnung',
                     'Bruttoleistung der Einheit',
                     'Rotordurchmesser der Windenergieanlage',
                     'Nabenhöhe der Windenergieanlage',
                     'Hersteller der Windenergieanlage']]
df_MaStR = df_MaStR.dropna()
df_MaStR = df_MaStR.reset_index()
df_MaStR = df_MaStR.drop('index', axis=1)

df_MaStR['Schlüssel'] = '-'

for i in tqdm(range(len(df_MaStR))):
    Index = (str(df_MaStR['Typenbezeichnung'][i]) + ', ' + 
            str(int(df_MaStR['Bruttoleistung der Einheit'][i])) + ', ' + 
            str(df_MaStR['Rotordurchmesser der Windenergieanlage'][i]) + ', ' +
            str(df_MaStR['Nabenhöhe der Windenergieanlage'][i]) + ', ' +
            str(df_MaStR['Hersteller der Windenergieanlage'][i]))
    df_MaStR.loc[i, 'Schlüssel'] = Index


df_MaStR = df_MaStR.set_index('Schlüssel')
df_MaStR

100%|██████████| 15722/15722 [00:03<00:00, 4656.66it/s]


,Typenbezeichnung,Bruttoleistung der Einheit,Rotordurchmesser der Windenergieanlage,Nabenhöhe der Windenergieanlage,Hersteller der Windenergieanlage
Schlüssel,,,,,
"FL1000, 1000, 54.0, 70.0, Fuhrländer AG",FL1000,1000.0,54.0,70.0,Fuhrländer AG
"N43, 600, 43.0, 60.0, Nordex Energy GmbH",N43,600.0,43.0,60.0,Nordex Energy GmbH
"V66, 1750, 66.0, 67.0, Vestas Deutschland GmbH",V66,1750.0,66.0,67.0,Vestas Deutschland GmbH
"V66, 1750, 66.0, 67.0, Vestas Deutschland GmbH",V66,1750.0,66.0,67.0,Vestas Deutschland GmbH
"MM82, 2000, 82.0, 59.0, Senvion Deutschland GmbH",MM82,2000.0,82.0,59.0,Senvion Deutschland GmbH
...,...,...,...,...,...
"V42 / 600, 600, 42.0, 53.0, Vestas Deutschland GmbH",V42 / 600,600.0,42.0,53.0,Vestas Deutschland GmbH
"LW18/80, 80, 18.0, 40.0, Lagerwey GmbH",LW18/80,80.0,18.0,40.0,Lagerwey GmbH
"Jacobs 43/600, 600, 43.0, 60.0, Jacobs Energie GmbH",Jacobs 43/600,600.0,43.0,60.0,Jacobs Energie GmbH


In [383]:
df_MaStR.insert(0, 'WTM', 'notfound')

keys = df_MaStR.index.to_list()
for key in tqdm(keys):
    try:
        model = df_final.loc[key]['Model']
        df_MaStR.loc[key, 'WTM'] = model
    except KeyError:
        continue

df_MaStR

  0%|          | 0/15722 [00:00<?, ?it/s]

100%|██████████| 15722/15722 [00:10<00:00, 1469.37it/s]


,WTM,Typenbezeichnung,Bruttoleistung der Einheit,Rotordurchmesser der Windenergieanlage,Nabenhöhe der Windenergieanlage,Hersteller der Windenergieanlage
Schlüssel,,,,,,
"FL1000, 1000, 54.0, 70.0, Fuhrländer AG",notfound,FL1000,1000.0,54.0,70.0,Fuhrländer AG
"N43, 600, 43.0, 60.0, Nordex Energy GmbH",Enercon E-40/6.44,N43,600.0,43.0,60.0,Nordex Energy GmbH
"V66, 1750, 66.0, 67.0, Vestas Deutschland GmbH",notfound,V66,1750.0,66.0,67.0,Vestas Deutschland GmbH
"V66, 1750, 66.0, 67.0, Vestas Deutschland GmbH",notfound,V66,1750.0,66.0,67.0,Vestas Deutschland GmbH
"MM82, 2000, 82.0, 59.0, Senvion Deutschland GmbH",notfound,MM82,2000.0,82.0,59.0,Senvion Deutschland GmbH
...,...,...,...,...,...,...
"V42 / 600, 600, 42.0, 53.0, Vestas Deutschland GmbH",IWT V42,V42 / 600,600.0,42.0,53.0,Vestas Deutschland GmbH
"LW18/80, 80, 18.0, 40.0, Lagerwey GmbH",notfound,LW18/80,80.0,18.0,40.0,Lagerwey GmbH
"Jacobs 43/600, 600, 43.0, 60.0, Jacobs Energie GmbH",Enercon E-40/6.44,Jacobs 43/600,600.0,43.0,60.0,Jacobs Energie GmbH


In [376]:
df_MaStR.to_excel('FinalResult.xlsx')